In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.pyplot as pyplt
import seaborn as sns

pd.options.mode.chained_assignment = None

# Preprocessing

In [ ]:
epl = pd.read_csv('/kaggle/input/epl-results-19932018/EPL_Set.csv')
epl

drop NA

In [ ]:
epl = epl[~epl.HTHG.isnull()]

In [ ]:
epl['HTHG'] = list(map(lambda x : int(x), epl.HTHG))
epl['HTAG'] = list(map(lambda x : int(x), epl.HTAG))

add Month

In [ ]:
epl['Month'] = list(map(lambda x : int(x.split('/')[1]), epl.Date))

add TotalGames

In [ ]:
group_total = epl.groupby('HomeTeam').Div.count().reset_index()
group_total.columns = ['HomeTeam','TotalGames']
epl = pd.merge(epl,group_total,how = 'outer')

In [ ]:
print('reversal :', np.round(np.mean(((epl.HTR == 'A') & (epl.FTR == 'H')) | ((epl.HTR == 'H') & (epl.FTR == 'A'))),2))

# Football Reversal Analysis

There are only 4% games that football reversal   

My Goal is that If my favorite team losed half-time, should I watch remainded half-time???   
I'm gonna analysis what is Characteristic of these games

In [ ]:
epl['reversal'] = 0
epl.loc[((epl.HTR == 'A') & (epl.FTR == 'H')) | ((epl.HTR == 'H') & (epl.FTR == 'A')), 'reversal'] = 1
epl

### Reversal Mean by Season

In [ ]:
group_season = epl.groupby('Season').reversal.mean().reset_index()
plt.figure(figsize=(19,5))
plt.bar(x = group_season.Season, height = group_season.reversal)
plt.show()

There is no dramatically increased point

---

### Reversal Mean by Month

In [ ]:
group_month = epl.groupby('Month').reversal.mean().reset_index()
plt.figure(figsize=(13,5))
plt.bar(x = group_month.Month, height = group_month.reversal)
plt.show()

At the start of League and the end of leaue have high point (April, September)

---

## EDA Only Reversal Game

In [ ]:
reversal_game = epl[epl.reversal == 1].reset_index(drop=True)

reversal_game['FinalWin'] = 0 ; reversal_game['FinalLose'] = 0
reversal_game.loc[reversal_game.FTR == 'H', 'FinalWin'] = reversal_game.HomeTeam
reversal_game.loc[reversal_game.FTR == 'A', 'FinalWin'] = reversal_game.AwayTeam
reversal_game.loc[reversal_game.FTR == 'H', 'FinalLose'] = reversal_game.AwayTeam
reversal_game.loc[reversal_game.FTR == 'A', 'FinalLose'] = reversal_game.HomeTeam

reversal_game['HDIFF'] = 0 ; reversal_game['FDIFF'] = 0
reversal_game['HDIFF'] = np.abs(reversal_game.HTHG - reversal_game.HTAG)
reversal_game['FDIFF'] = np.abs(reversal_game.FTHG - reversal_game.FTAG)
reversal_game['HowReversal'] = reversal_game.HDIFF + reversal_game.FDIFF

reversal_game

---

### FTR

In [ ]:
reversal_game.groupby('FTR').reversal.count()

Home > Away

---

### HDIFF FDIFF

In [ ]:
reversal_game.pivot_table(values = 'reversal', index = 'HDIFF', columns = 'FDIFF', aggfunc = 'count', fill_value = 0 )

In [ ]:
plt.figure(figsize=(18,5))
plt.plot(reversal_game.HowReversal, label = 'They catches by how many goal')
plt.plot(-reversal_game.HDIFF, label = 'Half-Time score difference')
plt.plot(reversal_game.FDIFF, label = 'Full-Time score difference')

plt.title('DIFFERENCE BETWEEN HALF-FULL')
plt.ylim(-5,10)
plt.legend(loc = 2)

plt.show()

It tends to be hard to reverse If team lose big

---

### Team

In [ ]:
group_FinalWin = reversal_game.groupby(['FinalWin']).reversal.count().reset_index()
group_FinalWin.columns = ['FinalWin','Count']
df_FinalWin = pd.merge(group_FinalWin, group_total, left_on='FinalWin', right_on = 'HomeTeam').drop(['HomeTeam'], axis = 1)
df_FinalWin['Mean'] = np.round(df_FinalWin.Count / df_FinalWin.TotalGames,3)
df_FinalWin.sort_values('Mean', ascending = False)[:20]

Traditional strong team (Arsnal, Man United, Tottenham, Chelsea)   
Promotion team (Ipswich, Sheffield United)   
have high trendency of reversal   

---
Then how to seperate these teams? By Total Games


In [ ]:
group_total.boxplot()
plt.show()

In [ ]:
group_total.quantile([0.25,0.5,0.75])

In [ ]:
q1 = float(group_total.quantile([0.25,0.5,0.75]).iloc[0])
q2 = float(group_total.quantile([0.25,0.5,0.75]).iloc[1])
q3 = float(group_total.quantile([0.25,0.5,0.75]).iloc[2])

In [ ]:
df_FinalWin['SeperateTeam'] = 0 
df_FinalWin.loc[df_FinalWin.TotalGames < q1, 'SeperateTeam'] = 1
df_FinalWin.loc[(df_FinalWin.TotalGames >= q1) & (df_FinalWin.TotalGames < q2), 'SeperateTeam'] = 2
df_FinalWin.loc[(df_FinalWin.TotalGames >= q2) & (df_FinalWin.TotalGames < q3), 'SeperateTeam'] = 3
df_FinalWin.loc[df_FinalWin.TotalGames >= q3, 'SeperateTeam'] = 4
df_FinalWin

In [ ]:
df1 = df_FinalWin.groupby('SeperateTeam').Count.sum().reset_index()
df2 = df_FinalWin.groupby('SeperateTeam').TotalGames.sum().reset_index()
df3 = pd.merge(df1,df2)
df3['Mean'] = df3.Count / df3.TotalGames
df3.plot.bar(x = 'SeperateTeam', y = 'Mean')
plt.show()

# Result

When your cheering team lose in half-time and you don't wanna watch the rest of match.   

Don't forgive. IF
1. on April or September.
2. traditional strong team or promotion team. 
3. Not lose big
4. Home Ground

Thank you :)